# Bias-Variance Decomposition for Regression Problems

In this example, we will see how to calculate the bias-variance decomposition for regression problems.

In [ ]:
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from mvtk.bias_variance import bias_variance_compute, bias_variance_mse
from mvtk.bias_variance.estimators import EstimatorWrapper

In [ ]:
random_state=123

## Load the example dataset

In [ ]:
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = housing.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

## Scikit-Learn Example

In [ ]:
from sklearn.linear_model import LinearRegression

from mvtk.bias_variance.estimators import SciKitLearnEstimatorWrapper

In [ ]:
model_scikit = LinearRegression()

## Need to instantiate a wrapper class for usage by the bias variance calculation

In [ ]:
model_scikit_wrapped = SciKitLearnEstimatorWrapper(model_scikit)

In [ ]:
# Use wrapped estimator
avg_loss, avg_bias, avg_var, net_var = bias_variance_compute(model_scikit_wrapped, X_train, y_train, X_test, y_test, iterations=10, 
                                                             random_state=random_state, decomp_fn=bias_variance_mse)

print(f'average loss: {avg_loss:10.8f}')
print(f'average bias: {avg_bias:10.8f}')
print(f'average variance: {avg_var:10.8f}')
print(f'net variance: {net_var:10.8f}')

## PyTorch Example

In [ ]:
import torch
import torch.nn as nn

from mvtk.bias_variance.estimators import PyTorchEstimatorWrapper

In [ ]:
X_train_torch = torch.FloatTensor(X_train.values)
X_test_torch = torch.FloatTensor(X_test.values)
y_train_torch = torch.FloatTensor(y_train).reshape(-1, 1)
y_test_torch = torch.FloatTensor(y_test).reshape(-1, 1)

In [ ]:
class ModelPyTorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(8, 24)
        self.linear2 = nn.Linear(24, 12)
        self.linear3 = nn.Linear(12, 6)
        self.linear4 = nn.Linear(6, 1)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        return x

In [ ]:
model_pytorch = ModelPyTorch()
optimizer = torch.optim.Adam(model_pytorch.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

## Need to instantiate a wrapper class for usage by the bias variance calculation

In [ ]:
def optimizer_generator(x):
    return torch.optim.Adam(x.parameters(), lr=0.001)

In [ ]:
model_pytorch_wrapped = PyTorchEstimatorWrapper(model_pytorch, optimizer_generator, loss_fn)

In [ ]:
# Use wrapped estimator
avg_loss, avg_bias, avg_var, net_var = bias_variance_compute(model_pytorch_wrapped, X_train_torch, y_train_torch, X_test_torch, y_test, 
                                                             iterations=10, random_state=random_state, decomp_fn=bias_variance_mse, 
                                                             fit_kwargs={'epochs': 100, 'batch_size': 300})

print(f'average loss: {avg_loss:10.8f}')
print(f'average bias: {avg_bias:10.8f}')
print(f'average variance: {avg_var:10.8f}')
print(f'net variance: {net_var:10.8f}')

## TensorFlow example

In [ ]:
import tensorflow as tf
from keras import initializers

from mvtk.bias_variance.estimators import TensorFlowEstimatorWrapper

In [ ]:
model_tensorflow = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0)),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer=initializers.glorot_uniform(seed=0)),
    tf.keras.layers.Dense(1, kernel_initializer=initializers.glorot_uniform(seed=0))
])

In [ ]:
model_tensorflow.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss='mean_absolute_error',
             metrics=['mean_squared_error'])

In [ ]:
model_tensorflow_wrapped = TensorFlowEstimatorWrapper(model_tensorflow)

In [ ]:
# Use wrapped estimator
avg_loss, avg_bias, avg_var, net_var = bias_variance_compute(model_tensorflow_wrapped, X_train, y_train, X_test, y_test, iterations=10, 
                                                             random_state=random_state, decomp_fn=bias_variance_mse, 
                                                             fit_kwargs={'epochs': 100, 'verbose': False}, 
                                                             predict_kwargs={'verbose': False})

print(f'average loss: {avg_loss:10.8f}')
print(f'average bias: {avg_bias:10.8f}')
print(f'average variance: {avg_var:10.8f}')
print(f'net variance: {net_var:10.8f}')

## We can run the same bias variance calculation in parallel for faster execution (in general for larger datasets and more intensive computations)

In [ ]:
from mvtk.bias_variance import bias_variance_compute_parallel

avg_loss, avg_bias, avg_var, net_var = bias_variance_compute_parallel(model_tensorflow_wrapped, X_train, y_train, X_test, y_test, 
                                                                      iterations=10, random_state=random_state, 
                                                                      decomp_fn=bias_variance_mse, 
                                                                      fit_kwargs={'epochs': 100, 'verbose': False}, 
                                                                      predict_kwargs={'verbose': False})

print(f'average loss: {avg_loss:10.8f}')
print(f'average bias: {avg_bias:10.8f}')
print(f'average variance: {avg_var:10.8f}')
print(f'net variance: {net_var:10.8f}')